In [1]:
import pathlib
import random

import tensorflow as tf
import numpy as np
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import matplotlib.pyplot as plt

In [2]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [3]:
checkpoint_path = 'training/'
imgroot_path = 'picture/train'
num_train = 800
num_val = 0
batch_size = 32
epochs = 1
IMG_HEIGHT = 192
IMG_WIDTH = 192

In [4]:
def load_image_label(path, label):
    image_raw = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image_raw, channels=3)
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    image /= 255.0
    return image, label

In [5]:
def load_image(path):
    image_raw = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image_raw, channels=3)
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    image /= 255.0
    return image

In [6]:
def plot_image(image):
    plt.figure(1)
    plt.imshow(image)
    plt.grid(False)
    plt.show()

In [7]:
data_path = pathlib.Path(imgroot_path)
all_image_paths = list(data_path.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths if str(path).endswith(".jpg")]
random.shuffle(all_image_paths)

image_count = len(all_image_paths)
label_names = sorted(item.name for item in data_path.glob('*/') if item.is_dir())
label_to_index = dict((name, index) for index, name in enumerate(label_names))
all_image_labels = [label_to_index[pathlib.Path(path).parent.name] for path in all_image_paths]

print(image_count)
num_val = image_count - num_train

1136


In [8]:
train_image_paths = all_image_paths[:num_train]
val_image_paths = all_image_paths[num_train:]

train_image_labels = all_image_labels[:num_train]
val_image_labels = all_image_labels[num_train:]

In [9]:
train_images = train_image_paths
val_images = val_image_paths

map(load_image, train_images)
map(load_image, val_images)

In [24]:
train_gen = ImageDataGenerator(rescale=1./255,
                               rotation_range=45,
                               width_shift_range=.15,
                               height_shift_range=.15,
                               horizontal_flip=True,
                               zoom_range=0.5)

val_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_data_gen = train_gen.flow(train_images)

In [27]:
train_data_gen = train_gen.flow_from_directory('picture/pre_trans',
                                               batch_size=1,
                                                shuffle=False,
                                                save_to_dir='picture/trans',
                                                save_prefix='trans_',
                                                save_format='jpg')

for i in range(8):
    next(train_data_gen)

Found 2 images belonging to 2 classes.


In [20]:
train_data_gen = train_gen.flow(train_images,
                                train_image_labels,
                                batch_size=batch_size)

val_data_gen = val_gen.flow(val_images,
                            val_image_labels,
                            batch_size=batch_size)

AttributeError: 'list' object has no attribute 'size'

In [ ]:
train_path_image_ds = tf.data.Dataset.from_tensor_slices((train_image_paths, train_image_labels))
val_path_image_ds = tf.data.Dataset.from_tensor_slices((val_image_paths, val_image_labels))

train_image_label_ds = train_path_image_ds.map(load_image_label)
val_image_label_ds = val_path_image_ds.map(load_image_label)

train_image_label_ds

In [ ]:
train_image_label_ds = train_image_label_ds.shuffle(buffer_size=image_count).repeat().batch(batch_size)
train_image_label_ds = train_image_label_ds.prefetch(buffer_size=AUTOTUNE)

val_image_label_ds = val_image_label_ds.shuffle(buffer_size=image_count).repeat().batch(batch_size)
val_image_label_ds = val_image_label_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
def create_model():
    mobile_net = tf.keras.applications.MobileNetV2(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), include_top=False)
    mobile_net.trainable=False

    model = models.Sequential([
        mobile_net,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.2),
        layers.Dense(8, activation = 'softmax')
        # layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        # layers.MaxPooling2D(),
        # layers.Dropout(0.2),
        # layers.Conv2D(32, 3, padding='same', activation='relu'),
        # layers.MaxPooling2D(),
        # layers.Conv2D(64, 3, padding='same', activation='relu'),
        # layers.MaxPooling2D(),
        # layers.Dropout(0.2),
        # layers.Flatten(),
        # layers.Dense(512, activation='relu'),
        # layers.Dense(16, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [ ]:
model = create_model()
model.summary()

In [ ]:
model_log = model.fit(train_image_label_ds, 
                      epochs=epochs, 
                      steps_per_epoch=num_train,
                      validation_data=val_image_label_ds,
#                       validation_steps=num_val)
 model.fit_generator(train_data_gen,
                    steps_per_epoch=num_train,
                    epochs=epochs,
                    validation_data=val_data_gen,
                    validation_steps=num_val)                      

In [ ]:
model = createmodel.save('training_res/model_generator.h5')

In [ ]:
acc = model_log.history['accuracy']
val_acc = model_log.history['val_accuracy']

loss = model_log.history['loss']
val_loss = model_log.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Accuracy')

plt.subplot(2, 1, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Loss')
plt.savefig('pic_A&L.jpg')
plt.show()